In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.reset_default_graph()
f = open('stock_dataset.csv')
df = pd.read_csv(f,encoding = 'utf-8')
data = np.array(df['Hprice'])
data = data[::-1]

# plot the data diagram
plt.figure()
plt.plot(data)
plt.show()
normalize_data = (data - np.mean(data))/np.std(data) #标准化
normalize_data = normalize_data[:,np.newaxis]

<Figure size 640x480 with 1 Axes>

In [2]:
normalize_data

array([[-1.59618755],
       [-1.59202899],
       [-1.58755926],
       ...,
       [ 1.60591603],
       [ 1.61341273],
       [ 1.56805529]])

In [3]:
normalize_data.shape

(6111, 1)

In [4]:
# 形成训练集
time_step = 20
rnn_unit = 10
batch_size = 60
input_size = 1
output_size = 1
lr = 0.0006
train_x, train_y = [], []
for i in range(len(normalize_data) - time_step - 1):
    x = normalize_data[i: i + time_step]
    y = normalize_data[i + 1: i + time_step + 1]
    train_x.append(x.tolist())
    train_y.append(y.tolist())
    

In [5]:
train_x

[[[-1.5961875451103236],
  [-1.5920289935368048],
  [-1.5875592578319344],
  [-1.5824482942200362],
  [-1.577073295587597],
  [-1.5723395248621856],
  [-1.5723300950400234],
  [-1.571226805847049],
  [-1.570132946476237],
  [-1.5689730783502895],
  [-1.567747201469207],
  [-1.5665213245881242],
  [-1.5659366756140696],
  [-1.5653520266400147],
  [-1.5647296583773114],
  [-1.5641355795810943],
  [-1.563532070962715],
  [-1.562985141277309],
  [-1.5634094832746068],
  [-1.5634094832746068]],
 [[-1.5920289935368048],
  [-1.5875592578319344],
  [-1.5824482942200362],
  [-1.577073295587597],
  [-1.5723395248621856],
  [-1.5723300950400234],
  [-1.571226805847049],
  [-1.570132946476237],
  [-1.5689730783502895],
  [-1.567747201469207],
  [-1.5665213245881242],
  [-1.5659366756140696],
  [-1.5653520266400147],
  [-1.5647296583773114],
  [-1.5641355795810943],
  [-1.563532070962715],
  [-1.562985141277309],
  [-1.5634094832746068],
  [-1.5634094832746068],
  [-1.5638715445605533]],
 [[-1.5875

In [6]:
my_x = np.asarray(train_x)
my_x.shape


(6090, 20, 1)

In [7]:
X = tf.placeholder(tf.float32, [None, time_step, input_size])
input = tf.reshape(X, [-1, input_size])
input

<tf.Tensor 'Reshape:0' shape=(?, 1) dtype=float32>

In [8]:
# Define Neural Network
X = tf.placeholder(tf.float32, [None, time_step, input_size])
Y = tf.placeholder(tf.float32, [None, time_step, output_size])

# input, output weights and biases
weights = {
    'in':tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out':tf.Variable(tf.random_normal([rnn_unit, output_size]))
}
biases = {
    'in':tf.Variable(tf.constant(0.1, shape = [rnn_unit,])),
    'out':tf.Variable(tf.constant(0.1, shape = [1,]))
}

# Define LSTM
def lstm(batch):      #参数：输入网络批次数目
    w_in=weights['in']
    b_in=biases['in']
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states
    
                     

In [13]:
def train_lstm():
    global batch_size
    with tf.variable_scope('forward'):
        pred,_=lstm(batch_size)
    #pred,_=lstm(batch_size)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    saver=tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #重复训练10000次
        for i in range(100):
            step=0
            start=0
            end=start+batch_size
            while(end<len(train_x)):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[start:end],Y:train_y[start:end]})
                start+=batch_size
                end=start+batch_size
                #每10步保存一次参数
                if step%10==0:
                    print(i,step,loss_)
                    print("保存模型：",saver.save(sess,'./stock.model'))
                step+=1


train_lstm()

0 0 8.237408
保存模型： ./stock.model
0 10 0.9133053
保存模型： ./stock.model
0 20 1.0654931
保存模型： ./stock.model
0 30 0.03221786
保存模型： ./stock.model
0 40 0.019085353
保存模型： ./stock.model
0 50 0.034296352
保存模型： ./stock.model
0 60 0.03609303
保存模型： ./stock.model
0 70 2.7165346
保存模型： ./stock.model
0 80 0.0659259
保存模型： ./stock.model
0 90 0.10618575
保存模型： ./stock.model
0 100 0.824358
保存模型： ./stock.model
1 0 1.5569105
保存模型： ./stock.model
1 10 0.044067744
保存模型： ./stock.model
1 20 0.06647241
保存模型： ./stock.model
1 30 0.13573258
保存模型： ./stock.model
1 40 0.0378038
保存模型： ./stock.model
1 50 0.017165141
保存模型： ./stock.model
1 60 0.023470981
保存模型： ./stock.model
1 70 0.96211714
保存模型： ./stock.model
1 80 0.08122556
保存模型： ./stock.model
1 90 0.10859957
保存模型： ./stock.model
1 100 0.3352492
保存模型： ./stock.model
2 0 0.6666381
保存模型： ./stock.model
2 10 0.03476432
保存模型： ./stock.model
2 20 0.051369287
保存模型： ./stock.model
2 30 0.030317001
保存模型： ./stock.model
2 40 0.011953915
保存模型： ./stock.model
2 50 0.00526846
保存模型： ./stock.mod

保存模型： ./stock.model
20 0 0.045398124
保存模型： ./stock.model
20 10 0.0070408774
保存模型： ./stock.model
20 20 0.00999722
保存模型： ./stock.model
20 30 0.0036014477
保存模型： ./stock.model
20 40 0.0021930821
保存模型： ./stock.model
20 50 0.0017319359
保存模型： ./stock.model
20 60 0.0039012074
保存模型： ./stock.model
20 70 0.046815947
保存模型： ./stock.model
20 80 0.005407008
保存模型： ./stock.model
20 90 0.002235181
保存模型： ./stock.model
20 100 0.022665206
保存模型： ./stock.model
21 0 0.04416812
保存模型： ./stock.model
21 10 0.0067650373
保存模型： ./stock.model
21 20 0.009375623
保存模型： ./stock.model
21 30 0.003430496
保存模型： ./stock.model
21 40 0.0021219384
保存模型： ./stock.model
21 50 0.0016048666
保存模型： ./stock.model
21 60 0.0036480534
保存模型： ./stock.model
21 70 0.04521187
保存模型： ./stock.model
21 80 0.0050917687
保存模型： ./stock.model
21 90 0.0022678392
保存模型： ./stock.model
21 100 0.021505978
保存模型： ./stock.model
22 0 0.04220197
保存模型： ./stock.model
22 10 0.0064152796
保存模型： ./stock.model
22 20 0.008793086
保存模型： ./stock.model
22 30 0.0032203451
保存模型

保存模型： ./stock.model
39 50 0.00092704146
保存模型： ./stock.model
39 60 0.0012466674
保存模型： ./stock.model
39 70 0.019287055
保存模型： ./stock.model
39 80 0.0035584236
保存模型： ./stock.model
39 90 0.00091621996
保存模型： ./stock.model
39 100 0.010732606
保存模型： ./stock.model
40 0 0.015203341
保存模型： ./stock.model
40 10 0.0029377106
保存模型： ./stock.model
40 20 0.0027041875
保存模型： ./stock.model
40 30 0.0010359474
保存模型： ./stock.model
40 40 0.0015035685
保存模型： ./stock.model
40 50 0.0009077065
保存模型： ./stock.model
40 60 0.001190835
保存模型： ./stock.model
40 70 0.01859361
保存模型： ./stock.model
40 80 0.0034872731
保存模型： ./stock.model
40 90 0.0008899291
保存模型： ./stock.model
40 100 0.010578429
保存模型： ./stock.model
41 0 0.014542532
保存模型： ./stock.model
41 10 0.002855826
保存模型： ./stock.model
41 20 0.002532881
保存模型： ./stock.model
41 30 0.0009958671
保存模型： ./stock.model
41 40 0.0014342825
保存模型： ./stock.model
41 50 0.00088376185
保存模型： ./stock.model
41 60 0.001138411
保存模型： ./stock.model
41 70 0.017916981
保存模型： ./stock.model
41 80 0.003412

保存模型： ./stock.model
58 80 0.0022884337
保存模型： ./stock.model
58 90 0.0008296846
保存模型： ./stock.model
58 100 0.009382889
保存模型： ./stock.model
59 0 0.0075452374
保存模型： ./stock.model
59 10 0.0020376132
保存模型： ./stock.model
59 20 0.00077790715
保存模型： ./stock.model
59 30 0.0005740196
保存模型： ./stock.model
59 40 0.0005411981
保存模型： ./stock.model
59 50 0.0004702159
保存模型： ./stock.model
59 60 0.0005491921
保存模型： ./stock.model
59 70 0.010512869
保存模型： ./stock.model
59 80 0.0022570784
保存模型： ./stock.model
59 90 0.0008609436
保存模型： ./stock.model
59 100 0.009300126
保存模型： ./stock.model
60 0 0.0073619103
保存模型： ./stock.model
60 10 0.002015481
保存模型： ./stock.model
60 20 0.00072312145
保存模型： ./stock.model
60 30 0.00055616634
保存模型： ./stock.model
60 40 0.00052642124
保存模型： ./stock.model
60 50 0.00046084975
保存模型： ./stock.model
60 60 0.00052880705
保存模型： ./stock.model
60 70 0.010269645
保存模型： ./stock.model
60 80 0.0022341511
保存模型： ./stock.model
60 90 0.00089644006
保存模型： ./stock.model
60 100 0.0092233475
保存模型： ./stock.model
61

77 100 0.008127581
保存模型： ./stock.model
78 0 0.0039485455
保存模型： ./stock.model
78 10 0.0020898741
保存模型： ./stock.model
78 20 0.0005057742
保存模型： ./stock.model
78 30 0.0005360092
保存模型： ./stock.model
78 40 0.00049741205
保存模型： ./stock.model
78 50 0.00057735
保存模型： ./stock.model
78 60 0.00044121454
保存模型： ./stock.model
78 70 0.010310658
保存模型： ./stock.model
78 80 0.0026699875
保存模型： ./stock.model
78 90 0.0006840997
保存模型： ./stock.model
78 100 0.0080804825
保存模型： ./stock.model
79 0 0.0025795763
保存模型： ./stock.model
79 10 0.002136061
保存模型： ./stock.model
79 20 0.00024275323
保存模型： ./stock.model
79 30 0.0004387273
保存模型： ./stock.model
79 40 0.00062035484
保存模型： ./stock.model
79 50 0.0005268819
保存模型： ./stock.model
79 60 0.00031060757
保存模型： ./stock.model
79 70 0.010825043
保存模型： ./stock.model
79 80 0.0036226972
保存模型： ./stock.model
79 90 0.0008646462
保存模型： ./stock.model
79 100 0.008188537
保存模型： ./stock.model
80 0 0.0035051878
保存模型： ./stock.model
80 10 0.0018505153
保存模型： ./stock.model
80 20 0.00034282258
保存模型： .

保存模型： ./stock.model
97 10 0.0017364745
保存模型： ./stock.model
97 20 0.00016602513
保存模型： ./stock.model
97 30 0.00020963371
保存模型： ./stock.model
97 40 0.0006410506
保存模型： ./stock.model
97 50 0.00037036795
保存模型： ./stock.model
97 60 0.0002666066
保存模型： ./stock.model
97 70 0.009522142
保存模型： ./stock.model
97 80 0.0053437934
保存模型： ./stock.model
97 90 0.0025467745
保存模型： ./stock.model
97 100 0.0091764415
保存模型： ./stock.model
98 0 0.0032866863
保存模型： ./stock.model
98 10 0.0016915182
保存模型： ./stock.model
98 20 0.00017608536
保存模型： ./stock.model
98 30 0.0002612362
保存模型： ./stock.model
98 40 0.0007212657
保存模型： ./stock.model
98 50 0.00042034
保存模型： ./stock.model
98 60 0.00021606828
保存模型： ./stock.model
98 70 0.015679033
保存模型： ./stock.model
98 80 0.0038037447
保存模型： ./stock.model
98 90 0.000587388
保存模型： ./stock.model
98 100 0.009213449
保存模型： ./stock.model
99 0 0.013079646
保存模型： ./stock.model
99 10 0.0018951637
保存模型： ./stock.model
99 20 0.00024186858
保存模型： ./stock.model
99 30 0.00030843684
保存模型： ./stock.model
99 40

In [14]:
def prediction():
    with tf.variable_scope('backward'):
        pred,_=lstm(1)
    #pred,_=lstm(1)    #预测时只输入[1,time_step,input_size]的测试数据
    saver=tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        #tf.reset_default_graph()
        #参数恢复
        module_file = tf.train.latest_checkpoint(base_path+'module2/')
        saver.restore(sess, module_file) 
        #取训练集最后一行为测试样本。shape=[1,time_step,input_size]
        prev_seq=train_x[-1]
        predict=[]
        #得到之后100个预测结果
        for i in range(100):
            next_seq=sess.run(pred,feed_dict={X:[prev_seq]})
            predict.append(next_seq[-1])
            #每次得到最后一个时间步的预测结果，与之前的数据加在一起，形成新的测试样本
            prev_seq=np.vstack((prev_seq[1:],next_seq[-1]))
        #以折线图表示结果
        plt.figure()
        plt.plot(list(range(len(normalize_data))), normalize_data, color='b')
        plt.plot(list(range(len(normalize_data), len(normalize_data) + len(predict))), predict, color='r')
        plt.show()
prediction()

NameError: name 'base_path' is not defined